In [2]:
import numpy as np
import xarray as xr 
from tqdm import tqdm
import matplotlib.pyplot as plt
import cmocean
import glob
import dask.array as da
import time
from numpy import nan
import pandas as pd
import seaborn as sns
import pymannkendall as mk

In [ ]:
# Figure 1a

import shapefile
import geopandas as gpd
import matplotlib.pyplot as plt
new = gpd.read_file('F:/!new added analysis/Export_Output.shp')
new['20Y'] = ["II-1" if x == 21 else x for x in new['20Y']] 
new['20Y'] = ["II-2" if x == 22 else x for x in new['20Y']] 
new['20Y'] = ["I" if x == 1 else x for x in new['20Y']] 
new['20Y'] = ["" if x == 0 else x for x in new['20Y']] 

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.stock_img()
ax.add_feature(ctp.feature.LAND, zorder = 1, linewidth=0.2, edgecolor='black', facecolor='white')
ax.gridlines(draw_labels=True,linestyle=":",linewidth=0.3,color='gray')

I = new[new['20Y'] == "I"]
ax.add_geometries(I['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#C1E0DB')
II1 = new[new['20Y'] == "II-1"]
ax.add_geometries(II1['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#FDDED7')
II2 = new[new['20Y'] == "II-2"]
ax.add_geometries(II2['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#F5BE8F')
III = new[new['20Y'] == ""]
ax.add_geometries(III['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='none')

PG = gpd.read_file('F:/!new added analysis/POP_GDP_watershed_output1.shp')

# 添加人口气泡
ax.scatter(x=PG['longitude'], y=PG['latitude'],
    s=np.sqrt(PG['population']/1e3)/10,  # 计算人口气泡大小
    c='darkblue',  # 单一颜色
    alpha=0.6, edgecolors='w', linewidth=0.3, transform=ccrs.PlateCarree()
)
# 添加人口图例
ax.legend_ = None  # 清空缓存
for population in [1e6, 1e7, 1e8]:
    ax.scatter([], [], c='darkblue', alpha=0.6,
               s=np.sqrt(population/1e3)/10,
               label=f'{population/1e6:.1f} M')
P_LEGEND = ax.legend(title='Population', loc='lower left', bbox_to_anchor=(0.04, 0.1), frameon=False)
ax.add_artist(P_LEGEND)

# 添加GDP气泡
ax.scatter(x=PG['longitude'], y=PG['latitude'],
    s=np.sqrt(PG['gdp_sum']/1e3)/2,  # 计算GDP气泡大小
    c='#B22222',  # 单一颜色
    alpha=0.6, edgecolors='w', linewidth=0.3, transform=ccrs.PlateCarree()
)
# 添加GDP图例
for GDP in [1e5, 1e6, 1e7]:
    ax.scatter([], [], c='#B22222', alpha=0.6,
               s=np.sqrt(GDP/1e3)/2,
               label=f'{GDP/1e6:.1f} M')
G_LEGEND = ax.legend(title='GDP (Million US$)', loc='lower left', bbox_to_anchor=(0.02, 0.1), frameon=False)
#ax.add_artist(G_LEGEND)

#ax.add_feature(ctp.feature.LAKES, alpha=0.5)
#ax.add_feature(ctp.feature.RIVERS, alpha=0.5, color='blue')
legend_labels=["I (35.4%)", "II-1 (44.2%)", "II-2 (20.4%)"]
L1 = mpatches.Rectangle((0, 0), 1, 1, facecolor="#C1E0DB")
L21 = mpatches.Rectangle((0, 0), 1, 1, facecolor="#FDDED7")
L22 = mpatches.Rectangle((0, 0), 1, 1, facecolor="#F5BE8F")
T_LEGEND = ax.legend([L1, L21, L22], legend_labels, title='Changing Style', loc='lower left', bbox_to_anchor=(0.02, 0.5), fancybox=True, frameon=False)
ax.add_artist(T_LEGEND)

plt.savefig('F:/!new added analysis/new figures/Figure1.png', dpi = 300)
plt.show()

In [ ]:
# Figure 1b

import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import matplotlib.patches as mpatches  # 新增，用于自定义图例

# ============ 1. 读取原有数据 ============
gdf = gpd.read_file('F:/!new added analysis/Export_Output3.shp')
basin_boundaries = gpd.read_file('F:/!new added analysis/diss1.shp')
gdf = gdf.to_crs(epsg=4326)
basin_boundaries = basin_boundaries.to_crs(epsg=4326)

land_ratio_col_map = {
    'Agriculture': 1,
    'Forest': 2,
    'Grassland': 3,
    'Shrubland': 4,
    'Wetland': 5,
    'Settlement': 6,
    'Bare areas': 7,
    'Water': 8,
    'Glacier': 9
}
gdf['land_code'] = gdf['第一大土地'].map(land_ratio_col_map)

# 假设列名里含有“城市占比”，并转换为数值
city_col_candidates = [col for col in gdf.columns if '城市占比（' in col]
if not city_col_candidates:
    raise ValueError("找不到含有‘城市占比’关键字的列，请检查列名。")
city_col_name = city_col_candidates[0]
gdf[city_col_name] = pd.to_numeric(gdf[city_col_name], errors='coerce')

# print("城市占比列统计：\n", gdf[city_col_name].describe())

# ============ 2. 读取 largest_basins.shp & 投影一致 ============
largest_basins = gpd.read_file('F:/!new added analysis/largest_basins_exclude.shp')
largest_basins = largest_basins.to_crs(epsg=4326)

# ============ 3. 创建绘图窗口 ============ 
fig, ax = plt.subplots(figsize=(12, 9), subplot_kw={'projection': ccrs.PlateCarree()})

# ============ 4. 添加底图元素 ============
ax.stock_img()  # 蓝色海洋底图
ax.add_feature(cfeature.LAND, facecolor='none', edgecolor='black', linewidth=0.5)
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)

# ============ 5. 设置网格线 ============
gl = ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=0.3,
    color='gray',
    linestyle=':'
)
gl.top_labels = True
gl.right_labels = True

# ============ 6. 绘制流域边界 & 主体地图 ============
AGRI = gdf[gdf['第一大土地'] == "Agriculture"]
ax.add_geometries(AGRI['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#F5BE8F')
BARE = gdf[gdf['第一大土地'] == "Bare areas"]
ax.add_geometries(BARE['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#FDDED7')
FORE = gdf[gdf['第一大土地'] == "Forest"]
ax.add_geometries(FORE['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#C1E0DB')
GLAC = gdf[gdf['第一大土地'] == "Glacier"]
ax.add_geometries(GLAC['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#9FDAF7')
GRAS = gdf[gdf['第一大土地'] == "Grassland"]
ax.add_geometries(GRAS['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#CCD376')
SHRU = gdf[gdf['第一大土地'] == "Shrubland"]
ax.add_geometries(SHRU['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#D9DEE7')
WATE = gdf[gdf['第一大土地'] == "Water"]
ax.add_geometries(WATE['geometry'], zorder = 1, crs=ccrs.PlateCarree(), linewidth=0.2, edgecolor='black', facecolor='#5CB0C3')
legend_labels=["Agriculture", "Bare areas", "Forest", "Grassland", "Shrubland", "Water"]
LAGRI = mpatches.Rectangle((0, 0), 1, 1, facecolor="#F5BE8F")
LBARE = mpatches.Rectangle((0, 0), 1, 1, facecolor="#FDDED7")
LFORE = mpatches.Rectangle((0, 0), 1, 1, facecolor="#C1E0DB")
LGLAC = mpatches.Rectangle((0, 0), 1, 1, facecolor="#9FDAF7")
LGRAS = mpatches.Rectangle((0, 0), 1, 1, facecolor="#CCD376")
LSHRU = mpatches.Rectangle((0, 0), 1, 1, facecolor="#D9DEE7")
LWATE = mpatches.Rectangle((0, 0), 1, 1, facecolor="#5CB0C3")
landuse_legend = ax.legend([LAGRI, LBARE, LFORE, LGLAC, LGRAS, LSHRU, LWATE], 
                           legend_labels, 
                           title='Land Use Type', 
                           loc='lower left', 
                           bbox_to_anchor=(0.02, 0.34), 
                           borderaxespad=0.5, 
                           fancybox=True,
                           frameon=False
)
# 将这个图例“添加”到绘图区
ax.add_artist(landuse_legend)

# ============ 7. 在地图上绘制 "ChenShi" 氣泡图（3 档离散） ============
# (1) 若 largest_basins 为 Polygon/MultiPolygon，需要先取质心
largest_basins['centroid'] = largest_basins.geometry.centroid

# (2) 将 "ChenShi" 列中空值填为 0（或按需处理）
largest_basins['ChenShi'] = largest_basins['ChenShi'].fillna(0)

# (3) 对 "ChenShi" 进行分箱，示例分成三档
bins = [0, 0.2, 0.5, np.inf]
labels = ['<0.5', '0.5~1.0', '>1.0']
largest_basins['ChenShi_cat'] = pd.cut(largest_basins['ChenShi'], bins=bins, labels=labels)

# (4) 定义每个档次的颜色和散点大小
color_map = {'<0.5': '#D0908F', '0.5~1.0': '#BE6C6D', '>1.0': '#AA3A49'}
size_map  = {'<0.5': 8, '0.5~1.0': 20, '>1.0': 40}

# (5) 分档绘制散点
for cat in labels:
    subset = largest_basins[largest_basins['ChenShi_cat'] == cat]
    ax.scatter(
        subset['centroid'].x,
        subset['centroid'].y,
        s=size_map[cat],
        color=color_map[cat],
        alpha=0.7,
        edgecolors='black',
        label=f'{cat}',   # 这一句决定散点对应的图例文字
        transform=ccrs.PlateCarree()
    )

# ============ 8. 其他修饰 & 显示图件 ============
# 让气泡散点的图例单独生成
# 注意：我们前面已经添加了一个 landcover_legend，这里再生成第二个图例
bubble_legend = ax.legend(
    title="Urbanization Rate (%)",
    loc='lower left',  # 可调整位置以避免与另一个图例重叠
    bbox_to_anchor=(0.02, 0.14),       # 相同左边缘，但往上偏移一些 (y=0.25)
    borderaxespad=0.5, # 可以根据需要调整图例在画布中的位置
    fancybox=True,
    frameon=False
)
ax.add_artist(bubble_legend)

ax.set_extent(
    [gdf.total_bounds[0], gdf.total_bounds[2],
     gdf.total_bounds[1], gdf.total_bounds[3]],
    ccrs.PlateCarree()
)

plt.savefig('F:/!new added analysis/new figures/Figure1b.png', dpi = 300)
plt.show()

In [ ]:
# Figure 2

import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.ticker as mticker
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.ticker as mticker
from matplotlib.ticker import MaxNLocator
import pandas as pd

# 1. 加载SHP文件
gdf = gpd.read_file('F:/!new added analysis/Export_Output2.shp')  # 替换为你的SHP文件路径


basin_boundaries = gpd.read_file('D:\\DATA\\BAS\\Major_hydrological_basins\\major_hydrobasins.shp')
basin_boundaries = basin_boundaries.to_crs(epsg=4326)  # 确保投影为 WGS84
basin_lines = basin_boundaries.boundary  # 获取边界线


# 2. 查看数据
print(gdf.head())  # 查看前几行数据
print(gdf.columns)  # 查看所有变量（列名）

# 3. 检查并设置投影（如果需要）
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # 如果没有投影，设置为WGS84
print(f"当前投影：{gdf.crs}")  # 打印当前投影信息

gdf['F18'] = gdf['F18'].replace(['NaN', ''], np.nan)
gdf['F19'] = gdf['F19'].replace(['NaN', ''],np.nan)

# 转换为数值类型
gdf['F18'] = pd.to_numeric(gdf['F18'], errors='coerce')
gdf['F19'] = pd.to_numeric(gdf['F19'], errors='coerce')

def combine_f18_f19(row):
    f18 = row['F18']
    f19 = row['F19']
    
    # 如果两者都缺值，设置为 0
    if np.isnan(f18) and np.isnan(f19):
        return 0
    # 如果 F18 缺值，使用 F19
    elif np.isnan(f18):
        return 0
    # 如果 F19 缺值，使用 F18
    elif np.isnan(f19):
        return 0
    # 如果两者都有值，取较大的值
    else:
        return 0

# 应用规则，创建新列 F18_F19
gdf['F18_F19'] = gdf.apply(combine_f18_f19, axis=1)


# 1. 创建图形和坐标轴
fig = plt.figure(figsize=(20, 16))
ax = plt.axes(projection=ccrs.PlateCarree())

# 2. 添加底图元素
ax.stock_img()  # 蓝色海洋底图
ax.add_feature(cfeature.LAND, facecolor='none', edgecolor='black', linewidth=0.5)
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)

# 3. 设置网格线
gl = ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=0.3,
    color='k'
)
gl.top_labels = True
gl.right_labels = True

# 4. 绘制F18_F19（不显示在图例中）
gdf.plot(
    column='F18_F19',
    cmap=ListedColormap(['white']),
    edgecolor='black',
    linewidth=0.5,
    ax=ax,
    legend=False
)

# 5. 自定义颜色映射
custom_cmap1 = LinearSegmentedColormap.from_list("terrain", ["#FFFFFF", '#F5BE8F'])  # 淡黄色
custom_cmap2 = LinearSegmentedColormap.from_list("terrain", ["#0baaa8", "#E0FFFF"])  # 紫色渐变 ["#EE82EE", "#E6E6FA"] 

# 6. 绘制F18（并添加colorbar）
f18_plot = gdf.plot(
    column='F18',
    cmap=custom_cmap2,
    ax=ax,
    legend=True,
    legend_kwds={
        'label': 'Decreasing trend',
        'orientation': 'horizontal',
        'shrink': 0.25,
        'pad': 0.15,  # 调整colorbar与地图的间距
        'aspect': 30,  # 控制colorbar的长宽比
        'anchor': (0.3, 3.35),
        'format': mticker.FormatStrFormatter('%.3f'),
        'ticks': MaxNLocator(nbins=5) 
    }
)

# 7. 绘制F19（并添加colorbar）
f19_plot = gdf.plot(
    column='F19',
    cmap=custom_cmap1,
    ax=ax,
    legend=True,
    legend_kwds={
        'label': 'Increasing trend',
        'orientation': 'horizontal',
        'shrink': 0.25,
        'pad': 0.15,  # 调整第二个colorbar的位置
        'aspect': 30,
        'anchor': (0.7,1.77),  # 尝试调整位置（可能效果有限）
        'format': mticker.FormatStrFormatter('%.3f'),
        'ticks': MaxNLocator(nbins=5) 
    }
)



# 8. 绘制流域边界
basin_lines.plot(
    ax=ax,
    color='black',
    linewidth=0.75
)

# plt.title('Map of F18 and F19 Values')

plt.rcParams['font.family'] = ['Calibri'] #Times New Roman
plt.rcParams['axes.unicode_minus'] = True
plt.tight_layout()
plt.show()

In [ ]:
# Figure 3

var_interval_list=['3D','5D','7D','10D','15D']
var_interval = var_interval_list[2]      ### 0 for 3D, 1 for 5D, 2 for 7D, 3 for 10D, 4 for 15D
data = xr.open_dataset('./20221020/Baisn_RF'+var_interval+'_stat.nc')
# look for the index of a basin
a = data['BASIN'].values
b = 'Nile'
idx = [idx0 for idx0, x in enumerate(a) if x == b]
idx[0]

In [88]:
# Figure 3a

RF_MAX = data['MAX_'+var_interval+'_SUM_RF']
RF_MIN = data['MIN_'+var_interval+'_SUM_RF']
diff = RF_MAX - RF_MIN
diff = xr.DataArray(diff,name='DIFF')   
diff.attrs['long_name']='MAX minus MIN'
diff.attrs['unit']='mm/s'

diff_std = diff.sel(year = slice(1850,2100)).std(('ensemble')).compute()
diff_mean = diff.sel(year = slice(1850,2100)).mean(('ensemble')).compute()
sn_diff = diff_mean[idx[0]]/diff_std[idx[0]]
sn_diff_rolling = sn_diff.rolling(year=10,center=True).mean()
fig,ax1 = plt.subplots(figsize=(8,6))
sn_diff_rolling.plot(label = 'SNR-DCR')
ax1.set_ylabel('SNR-DCR',fontsize = 12)
ax1.set_xlabel('')
#ax1.set_xlabel('Year',fontsize = 12)
ax1.tick_params(labelsize = 12)
plt.title('')
plt.ylim(4,7)
ax1.legend(loc = 'upper left',fontsize = 12)

diff_rolling = diff.rolling(year=10,center=True).mean()
diff_rolling_mean = diff_rolling.sel(year = slice(1850,2100)).mean(('ensemble')).compute()
ax2 = ax1.twinx()
diff_rolling_mean[idx[0]].plot(label = 'DCR_'+var_interval,color = 'r')
ax2.tick_params(labelsize = 12)
ax2.set_ylabel('DCR_'+var_interval,fontsize = 12)
ax2.set_xlabel('')
plt.xlim(1850,2100)
plt.ylim(0.01,0.03)
plt.title('')
#plt.title(label = data['BASIN'][idx[0]].values, fontsize = 12)
ax2.legend(loc = 'upper right',fontsize = 12)
plt.show()

In [89]:
# Figure 3b

DATE_MAX = data['MAX_'+var_interval+'_DATE']
DATE_MAX_rolling = DATE_MAX.rolling(year=10,center=True).mean()
DATE_MAX_rolling_mean = DATE_MAX_rolling.sel(year = slice(1850,2100)).mean(('ensemble')).compute()
fig,ax1 = plt.subplots(figsize=(8,6))
DATE_MAX_rolling_mean[idx[0]].plot(label = 'MAX_'+var_interval+'_DATE',color = 'g')
ax1.set_ylabel('MAX_'+var_interval+'_DATE',fontsize = 12)
#ax1.set_xlabel('Year',fontsize = 12)
ax1.tick_params(labelsize = 12)
plt.title('')
plt.ylim(200,250)
ax1.legend(loc = 'upper left',fontsize = 12)

DATE_MIN = data['MIN_'+var_interval+'_DATE']
DATE_MIN_rolling = DATE_MIN.rolling(year=10,center=True).mean()
DATE_MIN_rolling_mean = DATE_MIN_rolling.sel(year = slice(1850,2100)).mean(('ensemble')).compute()
ax2 = ax1.twinx()
DATE_MIN_rolling_mean[idx[0]].plot(label = 'MIN_'+var_interval+'_DATE',color = 'y')
ax2.tick_params(labelsize = 12)
ax2.set_ylabel('MIN_'+var_interval+'_DATE',fontsize = 12)
plt.xlim(1850,2100)
plt.ylim(80,100)
plt.title('')
#plt.title(label = data['BASIN'][idx[0]].values, fontsize = 12)
ax2.legend(loc = 'upper right',fontsize = 12)
plt.show()

In [ ]:
# Figure 3c and 3d

import cv2
ibasin = idx[0]
DATE_MAX = data['MAX_'+var_interval+'_DATE']
DATE_MIN = data['MIN_'+var_interval+'_DATE']
fig1,axs1 = plt.subplots(1,2,figsize=(16,6),sharex = False,sharey = False)
sns.distplot(DATE_MAX[ibasin,:,0:50],
            vertical = False,
            color = 'orange',
            norm_hist = True,
            hist_kws={'linewidth':0},
            kde_kws = {'linewidth':0.8, 'color':'r'},
            label = '1850-1900',
            ax = axs1[0])
sns.distplot(DATE_MAX[ibasin,:,200:251],
            vertical = False,
            color = 'green',
            norm_hist = True,
            hist_kws={'linewidth':0},
            kde_kws = {'linewidth':0.8, 'color':'b'},
            label = '2050-2100',
            ax = axs1[0])
axs1[0].set_xlabel('Order of day in a year',fontsize = 12)
axs1[0].set_ylabel('Density',fontsize = 12)
axs1[0].set_title('MAX_'+var_interval+'_DATE' + ' for ' + data['BASIN'].values[ibasin],fontsize = 12)
axs1[0].tick_params(labelsize = 12)
axs1[0].legend(loc = 'upper left',fontsize = 12)
sns.distplot(DATE_MIN[ibasin,:,0:50],
            vertical = False,
            color = 'orange',
            norm_hist = True,
            hist_kws={'linewidth':0},
            kde_kws = {'linewidth':0.8, 'color':'r'},
            label = '1850-1900',
            ax = axs1[1])
sns.distplot(DATE_MIN[ibasin,:,200:251],
            vertical = False,
            color = 'green',
            norm_hist = True,
            hist_kws={'linewidth':0},
            kde_kws = {'linewidth':0.8, 'color':'b'},
            label = '2050-2100',
            ax = axs1[1])
axs1[1].set_xlabel('Order of day in a year',fontsize = 12)
axs1[1].set_ylabel('Density',fontsize = 12)
axs1[1].set_title('MIN_'+var_interval+'_DATE' + ' for ' + data['BASIN'].values[ibasin],fontsize = 12)
axs1[1].tick_params(labelsize = 12)
axs1[1].legend(loc = 'upper left',fontsize = 12)

In [ ]:
# Figure 3e

iceduration_mod = diff    
pi_std = iceduration_mod.sel(year = slice(1850,1900)).std(('year','ensemble')).compute()
pi_mean = iceduration_mod.sel(year = slice(1850,1900)).mean(('year','ensemble')).compute()
comfort_line = pi_mean - pi_std * 2
## comfort_line cannot be negative
comfort_line = comfort_line.where(comfort_line >0, 0)

iceduration_mod_ensmean = iceduration_mod.mean('ensemble').compute()  ###
iceduration_2010 = iceduration_mod[:,150:160,...].where(iceduration_mod[:,150:160,...] > 0 )
iceduration_2010_ann = iceduration_2010.count('year').sum('ensemble').compute()
iceduration_2010_ann_mask = (iceduration_2010_ann > (10 * iceduration_mod.shape[0] - 10))
iceduration_2010_int_mask = xr.ufuncs.logical_and(iceduration_2010_ann <= (10 * iceduration_mod.shape[0] - 10), iceduration_2010_ann > 0 )

mask_window_start = xr.where( (iceduration_mod_ensmean.sel(year = slice(1901,2100)) - 2 * iceduration_mod.sel(year = slice(1901,2100)).std('ensemble') ) <= comfort_line, 1, 0)
mask_window_end   = xr.where( (iceduration_mod_ensmean.sel(year = slice(1901,2100)) + 2 * iceduration_mod.sel(year = slice(1901,2100)).std('ensemble') ) <= comfort_line, 1, 0)
mask_window_start = mask_window_start.where(iceduration_2010_ann > 0 ).compute()
mask_window_end = mask_window_end.where(iceduration_2010_ann > 0 ).compute()

In [8]:
def adap_years(mask_array_start,mask_array_end):
    if np.isnan(mask_array_start).any():
        return nan
    else:
        start = consecutive_idx(mask_array_start)
        end   = consecutive_idx(mask_array_end)
        if end == 2100:
            adap_window = 200 #149
        else:
            adap_window = end - start + 1
        return adap_window
#
def signal_emerge(mask_array_start,mask_array_end):
    if np.isnan(mask_array_start).any():
        return nan
    else:
        start = consecutive_idx(mask_array_start)
        return start
#
def signal_termin(mask_array_start,mask_array_end):
    if np.isnan(mask_array_end).any():
        return nan
    else:
        end = consecutive_idx(mask_array_end)
        return end

In [9]:
# iceduraiton_sig_emerge is the time of emergence of anthropogenic signal,
# iceduration_sig_termin is the time of emergence of no-analogue conditions
iceduration_sig_emerge = xr.apply_ufunc(signal_emerge,
                                     mask_window_start,
                                     mask_window_end,
                                     input_core_dims=[['year'],['year']],
                                     vectorize= True)
iceduration_sig_termin = xr.apply_ufunc(signal_termin,
                                     mask_window_start,
                                     mask_window_end,
                                     input_core_dims=[['year'],['year']],
                                     vectorize= True)
iceduration_adap_window = xr.apply_ufunc(adap_years,
                                     mask_window_start,
                                     mask_window_end,
                                     input_core_dims=[['year'],['year']],
                                     vectorize= True)

In [10]:
'''
update universal settings for fonts
'''
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    'pdf.fonttype': 42,
    'font.size':5,
    'axes.linewidth':0.3,
    'xtick.major.width':0.5,
    'xtick.minor.width':0.3,
    'ytick.major.width':0.5,
    'ytick.minor.width':0.3})

In [13]:
from matplotlib.patches import Patch
def line_span_hist_plot(input_data = iceduration_mod, input_data_std = pi_std,
                        time_emerge = iceduration_sig_emerge, time_termin = iceduration_sig_termin,
                        ibasin = idx[0], ###
                        left = 0, width = 0.5, width_h = 0.2,
                        bottom = 0, height = 0.5,
                        mother_fig = None):

    input_data = input_data[ibasin,:,:].compute()
    bottom_h = left_h = left + width
    rect_scatter = [left, bottom, width, height]
    rect_histy = [left_h, bottom, width_h, height]
    ax = mother_fig.add_axes(rect_histy,
             frame_on = False)
    y_upper = input_data[:,:].max()+0.0001   #
    y_lower = input_data[:,:].min()-0.001   #
    ax.set_ylim(y_lower,y_upper)
    ax.set_xlim(0,0.05)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.axhline(input_data[:,0:50].mean() + input_data_std[ibasin] * 2,
               xmin = 0,
               xmax = 0.2,
               c = 'green',
               ls = '-.' ,
              lw = 0.8)
    ax.axhline(input_data[:,0:50].mean() - input_data_std[ibasin] * 2,
               xmin = 0,
               xmax = 0.2,
               c = 'green',
               ls = '-.' ,
              lw = 0.8)


    axs = fig.add_axes(rect_scatter)
    axs.fill_betweenx(y = np.arange(-3,142),
                      x1 = time_emerge[ibasin],
                      x2 = time_termin[ibasin],
                      color = 'Gray',
                      alpha = 0.4,
                      zorder = 1)
    input_data[:,:].mean('ensemble').plot(ax = axs,
                                              color = 'red',
                                              lw = 0.8,
                                              zorder = 10)
    y_mean = input_data.mean('ensemble')
    y_std  = input_data.std('ensemble')
    axs.fill_between(x = y_mean.year,
                    y1 = y_mean - 2* y_std,
                     y2 = y_mean + 2 * y_std,
                    color = 'Blue',
                    alpha = 0.2,
                    label = '2$\sigma$')

    for i in np.arange(input_data.shape[0]):
        axs.scatter(input_data.year,
                    input_data[i,:],
                    c = 'royalblue',
                    alpha = 0.4,
                    s = 0.8,
                    edgecolor = 'none',
                    zorder = 3)
    axs.axhline(input_data[:,0:50].mean() + input_data_std[ibasin] * 2,
               c = 'green',
               ls = '-.'  ,
              lw = 0.8)
    axs.axhline(input_data[:,0:50].mean() - input_data_std[ibasin] * 2,
               c = 'green',
               ls = '-.' ,
              lw = 0.8)

    axs.set_ylim(y_lower,y_upper)
    axs.set_xlim(1850,2100)
    axs.set_ylabel('DCR',fontsize = 12)
    axs.set_xlabel('Year',fontsize = 12)
    axs.set_title(data['BASIN'].values[ibasin],fontsize = 12)
    axs.tick_params(labelsize = 8)


    axs.text(time_emerge[ibasin] - 12,
            68,
            "$t_{0}$",
            color = 'k',
            fontsize = 12
            )
    axs.text(time_termin[ibasin] + 1,
            68,
            "$t_{1}$",
            color = 'k',
            fontsize = 12
            )

    custom_legend = [plt.Line2D([0],[0],
                               marker='o',
                               color = 'w',
                               markerfacecolor = 'royalblue',
                               label = 'DCR in each ensemble',
                               markersize = 2),
                     plt.Line2D([0],[0],
                                color='red',
                                lw = 1,
                                label = 'Ensemble mean of DCR'),
                     Patch(facecolor='blue',
                           alpha = 0.15,
                          label = '2$\sigma$ among ensembles'),
                     plt.Line2D([0],[0],
                                color='green',
                                label = 'Natural variablity range in 1850-1900',
                                linestyle = '-.',
                               lw = 1) ]
    axs.legend(handles = custom_legend,
                loc = (0.02,0.01),
              fontsize = 8
              )

In [ ]:
''' 
in order to get information about warming amplitude, we add the temperature anomaly relative to the climatological mean of 1850-1950 besides the years
the temperature anomaly is from 
df = pd.read_csv('.../trefht_glob_mean.csv')
df['Temp_anomaly'] = df.Temperature_glob_mean - df[df.year < 1950].Temperature_glob_mean.mean()
in which global mean is from TREFHT, which is 2 m temperature including on land and ocean.
'''
cm = 1/2.54  # centimeters in inches
fig = plt.figure(figsize = (40 * cm ,25 * cm))

line_span_hist_plot(mother_fig = fig,left=0.15, bottom=0.53, width=0.3, width_h=0.1, height=0.35)
mother_fig=fig
#rect_histy = [left_h, bottom, width_h, height]

ibasin = idx[0]
input_data = iceduration_mod
input_data = input_data[ibasin,:,:].compute()
y_upper = input_data[:,:].max()+0.0001    # 
y_lower = input_data[:,:].min()-0.001    #
rect_histy = [0.45, 0.53, 0.05, 0.35]
ax = mother_fig.add_axes(rect_histy,
             frame_on = False)
ax.set_ylim(y_lower,y_upper)  ###
ax.set_xticks([])
ax.set_yticks([])

sns.distplot(input_data[:,0:50],
            vertical = True,
            color = 'orange',
            norm_hist = True,
            hist_kws={'linewidth':0},
            kde_kws = {'linewidth':0.8})
plt.show(fig)

In [ ]:
# Figure 3f

# ridge plot for max-min
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
x = diff_all.T.reshape((90*interval*8,1))[:,0]
y = np.tile(["0.5 °C","1.0 °C","1.5 °C","2.0 °C","2.5 °C","3.0 °C","3.5 °C","4.0 °C"], 90*interval)
df = pd.DataFrame(dict(x=x, y=y))
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(df, row="y", hue="y", aspect=12, height=.5, palette=pal)

# Draw the densities in a few steps
g.map(sns.kdeplot, "x",
      bw_adjust=.5, clip_on=False,
      fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, "x", clip_on=False, color="w", lw=2, bw_adjust=.5)

# passing color=None to refline() uses the hue mapping
#g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)
g.map(plt.axhline, y=0, linewidth=2, linestyle="-", color=None, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "x")

# Set the subplots to overlap
g.fig.subplots_adjust(hspace=-.25)

# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(xlabel="DCR")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)